In [ ]:
# 문제 정의

In [1]:
# 라이브러리 불러오기
import pandas as pd

train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")


In [2]:
# EDA
train.shape , test.shape # 1

((20758, 18), (13840, 17))

In [18]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20758 entries, 0 to 20757
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              20758 non-null  int64  
 1   Gender                          20758 non-null  object 
 2   Age                             20758 non-null  float64
 3   Height                          20758 non-null  float64
 4   Weight                          20758 non-null  float64
 5   family_history_with_overweight  20758 non-null  object 
 6   FAVC                            20758 non-null  object 
 7   FCVC                            20758 non-null  float64
 8   NCP                             20758 non-null  float64
 9   CAEC                            20758 non-null  object 
 10  SMOKE                           20758 non-null  object 
 11  CH2O                            20758 non-null  float64
 12  SCC                             

In [10]:
# train.describe()
# test.isnull().sum().sum()

In [17]:
# train.describe()
# train.describe(include='O')
# test.describe()
# train.info()
train['NObeyesdad'].value_counts()

NObeyesdad
Obesity_Type_III       4046
Obesity_Type_II        3248
Normal_Weight          3082
Obesity_Type_I         2910
Insufficient_Weight    2523
Overweight_Level_II    2522
Overweight_Level_I     2427
Name: count, dtype: int64

### id 제외
id 학습에서 제외

In [2]:
# id칼럼 제외
train = train.drop(columns=['id'])
test = test.drop(columns=['id'])

### 여기서 틀림

In [5]:
#y값 분리
# target = train.pop("NObeyesdad")
# dtype -> category
target = target.astype('category')
target.info()

<class 'pandas.core.series.Series'>
RangeIndex: 20758 entries, 0 to 20757
Series name: NObeyesdad
Non-Null Count  Dtype   
--------------  -----   
20758 non-null  category
dtypes: category(1)
memory usage: 20.7 KB


In [ ]:
# train/test object -> category

In [6]:
train['Gender'] = train['Gender'].astype('category')
train['family_history_with_overweight'] = train['family_history_with_overweight'].astype('category')
train['FAVC'] = train['FAVC'].astype('category')
train['CAEC'] = train['CAEC'].astype('category')
train['SMOKE'] = train['SMOKE'].astype('category')
train['SCC'] = train['SCC'].astype('category')
train['CALC'] = train['CALC'].astype('category')
train['MTRANS'] = train['MTRANS'].astype('category')

test['Gender'] = test['Gender'].astype('category')
test['family_history_with_overweight'] = test['family_history_with_overweight'].astype('category')
test['FAVC'] = test['FAVC'].astype('category')
test['CAEC'] = test['CAEC'].astype('category')
test['SMOKE'] = test['SMOKE'].astype('category')
test['SCC'] = test['SCC'].astype('category')
test['CALC'] = test['CALC'].astype('category')
test['MTRANS'] = test['MTRANS'].astype('category')

In [10]:
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20758 entries, 0 to 20757
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   Gender                          20758 non-null  category
 1   Age                             20758 non-null  float64 
 2   Height                          20758 non-null  float64 
 3   Weight                          20758 non-null  float64 
 4   family_history_with_overweight  20758 non-null  category
 5   FAVC                            20758 non-null  category
 6   FCVC                            20758 non-null  float64 
 7   NCP                             20758 non-null  float64 
 8   CAEC                            20758 non-null  category
 9   SMOKE                           20758 non-null  category
 10  CH2O                            20758 non-null  float64 
 11  SCC                             20758 non-null  category
 12  FAF               

## ''

In [ ]:
## 검증데이터 분할

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_val, y_train, y_val = train_test_split(
    train,
    target,
    test_size=0.2,
    random_state=0
)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((16606, 16), (4152, 16), (16606,), (4152,))

In [ ]:
## 모델 학습 및 평가

In [12]:
import lightgbm as lgb
# 학습
lgbmc = lgb.LGBMClassifier(ramdom_state=0 , verbose=-1)
lgbmc.fit(X_train,y_train)
#검증
pred = lgbmc.predict(X_val) #예측 y값 라벨만
pred_proba = lgbmc.predict_proba(X_val) #예측 확률값

In [ ]:
# verbose : 학습과정 출력여부 (-1 : 출력안함)
# 0: 경고 수준 출력 , 1: 인반정보 출력 , >1 : 자세한 정보 출력

In [16]:
# 평가(roc_auc ,accuracy , f1 )
from sklearn.metrics import roc_auc_score, f1_score , accuracy_score
roc_auc_ovr = roc_auc_score(y_val,pred_proba,multi_class='ovr') # 예측 확률값[오답예측확률, 정답예측확률]
roc_auc_ovo = roc_auc_score(y_val,pred_proba,multi_class='ovo')
accuracy = accuracy_score(y_val,pred)    # pred 예측 y값
f1= f1_score(y_val,pred,average='macro') # pred 예측 y값

print('roc_auc_ovr : ',  roc_auc_ovr)
print('roc_auc_ovo : ',  roc_auc_ovo)
print('accuracy : ',accuracy)
print('f1 : ', f1)


roc_auc_ovr :  0.9888169248146651
roc_auc_ovo :  0.9874571729788963
accuracy :  0.9051059730250481
f1 :  0.8949765427420396


In [24]:
# 예측 팡리 제출
pred_submit = lgbmc.predict(test)
pred_submit

submit_csv = pd.DataFrame({'pred' : pred_submit})
submit_csv.to_csv("reusjasdhf.csv",index=False)